In [5]:
import math
import cv2
import numpy as np
import mediapipe as mp

In [6]:
# Initializing mediapipe pose class.
mp_pose = mp.solutions.pose

# Setting up the Pose function.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

# Initializing mediapipe drawing class, useful for annotation.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [7]:
def calculateAngle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [8]:
cap = cv2.VideoCapture("KneeBendVideo.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

bent_counter = 0
rep_count = 0
threshold = 140
holding_timer_limit = 8
feedback=""

video_writer = cv2.VideoWriter(('output.mp4'), cv2.VideoWriter_fourcc(*'XVID'), fps, (w, h)) 
 
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
  while cap.isOpened():
    success, frame = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
       #If loading a video, use 'break' instead of 'continue'.
      continue
    
    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            left_knee_angle = calculateAngle(left_hip,left_knee,left_ankle)
            print(left_knee_angle)
            
            # Visualize angle
            cv2.putText(image, str(left_knee_angle), 
                           tuple(np.multiply(left_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
    except:
            pass
    

    # Render detections
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
    

    time = (1 / fps) * bent_counter

    if left_knee_angle > threshold:
        bent_counter = 0
    
    if left_knee_angle < threshold:
        bent_counter += 1
    
    if time == holding_timer_limit:
        rep_count += 1

    elif time < 8 and time >1 :
        feedback = 'Keep Your Knee Bent'

    elif time> 8.5:
        feedback = "Relax"
        
    else:
        feedback=""
   
        

    # cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
    
    cv2.putText(image, 'REP COUNT', (10,75),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0),2)
    
    cv2.putText(image, 'FEEDBACK', (10,100), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0),2)
    
    cv2.putText(image, 'TIMER', (10,125), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0),2)

    cv2.putText(image, str(rep_count),(110,75),cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255),2)
    
    if feedback == 'Keep Your Knee Bent':
        cv2.putText(image, feedback, (100,100), cv2.FONT_HERSHEY_PLAIN, 1, (255,0,0),2)
        
    else: 
        cv2.putText(image, feedback, (100,100), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0),2)

    cv2.putText(image, str(round(time,2)), (100,125), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255),2) 
    
    

    cv2.imshow('Knee Bend Excercise', image)
    
    video_writer.write(image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
video_writer.release()


154.9475318338754
172.2801618946054
176.06442398962278
166.1585233708121
154.49686785438314
162.9491979941186
166.768986453771
165.8746256199424
168.6185902205039
169.3687608723595
166.60912505060398
162.20690318012845
161.62638340959722
161.77138339270792
162.63230635456662
160.77748629496932
152.63513732127115
152.17795539282437
153.27386185517125
148.51074049299044
148.53252672695257
147.40611539079316
151.62429063717863
160.33038434989058
165.33634853865934
167.92716237648492
163.0631806995731
169.00364632410972
177.68434720210817
179.5799224543755
177.78544888294448
176.90765903585879
178.4345244786799
176.13940212981555
175.23426876048555
169.63125298219182
171.35395100453675
171.70282269319293
177.89998259273608
178.3696527681438
178.62814037569152
174.80104117519855
174.83545407751396
164.906006877385
167.46147924987653
172.08999299901228
166.85346202298524
175.54638909224192
167.5492625664642
165.26139089059268
154.92563524531187
152.29939853090005
154.6126700244495
159.882685